<a href="https://colab.research.google.com/github/UNIST-LIM-Lab-course/bert_fine_assignmnet-JJukE/blob/main/BERT_fine_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BERT Assignment 

The task objective is to code sentiment analysis by BERT 

* An assignment part is denoted by (Assingment) 

* Grading criteria: Points are given if all your code in this notebook is runnable and the final acc is upper than 0.5

* Points are not given if the testing cell at the end of the notebook is modified or extra cells (including text) are added after the last cell. Do not change **epochs** for testing efficiently.

* Testing your model with the testing cell is recommended. 

* Please do not re-use the code from the example code. You have to write the code yourself.


## Assignmnet List 
* (Assignment) 1.2 Upload tokenizer 
* (Assignment) 1.3 Make your Dataset by BertDataset function 
* (Assignment) 2.1 Make BERT Model by pretrained_model
* (Assignment) 3.1 Freeze parameters 
* (Assignment) 3.2 Make finetune function 

#1. Preparing Data



In [2]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 10.0 MB/s 
     |████████████████████████████████| 163 kB 65.4 MB/s 
     |████████████████████████████████| 7.6 MB 56.6 MB/s 


In [3]:
import pandas as pd
import numpy as np
import transformers
import torch
import torch.nn as nn

import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from tqdm import tqdm

## 1.1 Make BertDataset

In [47]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class BertDataset(Dataset):
    def __init__(self, tokenizer,max_length):
        super(BertDataset, self).__init__()
        self.train_csv=pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)
        self.tokenizer=tokenizer
        self.target=self.train_csv.iloc[:,1]
        self.max_length=max_length
        
    def __len__(self):
        return len(self.train_csv)
    
    def __getitem__(self, index):
        
        text1 = self.train_csv.iloc[index,0]
        inputs = self.tokenizer.encode_plus(
            text1 ,
            None,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            max_length=self.max_length,
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'target': torch.tensor(self.train_csv.iloc[index, 1], dtype=torch.long)
            }


## (Assignment) 1.2 Upload tokenizer 
* Fill out `tokenizer` by using BertTokenizer of "prajjwal1/bert-tiny" #(TODO:Assignment).


In [48]:
tokenizer = transformers.BertTokenizer.from_pretrained("prajjwal1/bert-tiny", do_lower_case=True) #(TODO:Assignment)

  0%|          | 0/217 [12:12<?, ?it/s]


## (Assignment) 1.3 Make your Dataset by `BertDataset`
* Fill out `dataset` by using BertDataset` 
* Fill out `dataloader` by using `DataLoader` of torch.utils.data
* Set max_length=100 
* Set batch_size=32 

In [49]:
dataset= BertDataset(tokenizer, 100) #(TODO:Assignment)
dataloader= torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True) #(TODO:Assignment)

# 2. Construction your Model 

## (Assignment) 2.1 Make BERT Model by pretrained_model 
* Use the pretrained_model `prajjwal1/bert-tiny`
* Simplest example code is below. But you can design BERT Model appropriately.
* Do not stack more than three layers. The grading time should not exceed 3 minutes.

In [55]:
# Example code
import torch.nn as nn
class BERT(nn.Module):
    def __init__(self):
        super(BERT, self).__init__()
        self.bert_model = transformers.BertModel.from_pretrained("prajjwal1/bert-tiny")
        self.out = nn.Linear(128, 1)
        
    def forward(self,ids,mask,token_type_ids):
        _,o2= self.bert_model(ids,attention_mask=mask,token_type_ids=token_type_ids, return_dict=False)
        out= self.out(o2)
        return out
    
model=BERT()

Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [56]:
import torch.nn as nn
class BERT(nn.Module):
    def __init__(self):
        super(BERT, self).__init__()
        self.bert_model = transformers.BertModel.from_pretrained("prajjwal1/bert-tiny")
        #(TODO:Assignment) 1) Define your BERT Model
        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(128, 32)
        self.fc2 = nn.Linear(32, 1)
        self.elu = nn.ELU()

    def forward(self,ids,mask,token_type_ids):
        _,o2= self.bert_model(ids,attention_mask=mask,token_type_ids=token_type_ids, return_dict=False)
        #(TODO:Assignment) 2) Forward your BERT Model
        out = self.fc1(self.dropout(o2))
        out = self.elu(out)
        out = self.fc2(self.dropout(out))
        return out
    
model=BERT()

Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# 3. Train your Moel 

In [57]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer= optim.Adam(model.parameters(),lr= 0.0001)

## (Assignment) 3.1 Freeze parameters 
* Fill out `#(TODO:Assignment)` to freeze parameters of BERT. 
* We should update prameters of non-BERT

In [58]:
for param in model.bert_model.parameters():
  param.requires_grad = False #(TODO:Assignment) 1) Set requires_grad to be False

In [59]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,161 trainable parameters


## (Assignment) 3.2 Make finetune function 
* Fill out `#(TODO:Assignment)` to make `def fineture`

1) Define loop by dataloader 

2) Set the grad of optimzer to be zero

3) Calculate the loss_fn(output, label) and the backward of the loss_fn

4) Estimate the everage accuracy of the model for each epoch 

5) Estimate the total everage accuracy of the model

In [60]:
def finetune(epochs,dataloader,model,loss_fn,optimizer):
    model.train()
    for  epoch in range(epochs):
        acc = 0 
        N = 0 
        #(TODO:Assignment) 1) Define loop by dataloader
        loop = enumerate(tqdm(dataloader))
        for batch, dl in loop:
          ids=dl['ids']
          token_type_ids=dl['token_type_ids']
          mask= dl['mask']
          label=dl['target']
          label = label.unsqueeze(1)

          #(TODO:Assignment) 2) Set the grad of optimzer to be zero
          optimizer.zero_grad()
          output=model(
                ids=ids,
                mask=mask,
                token_type_ids=token_type_ids)
          label = label.type_as(output)
          
          #(TODO:Assignment) 3) Calculate the loss_fn(output, label) and the backward of the loss_fn
          loss = loss_fn(output, label)
          loss.backward()
          optimizer.step()

          #(TODO:Assignment) 4) Estimate the average accuracy of the model for each epoch
          y_pred = torch.where(output >= 0, 1, 0).type(torch.float32).flatten().tolist()
          y_true = label.type(torch.float32).flatten().tolist()

          cnt = 0
          for i in range(len(y_true)):
            if y_pred[i] == y_true[i]:
              cnt += 1

          acc += cnt/len(y_true)
          N += 1
    
    #(TODO:Assignment) 5) Estimate the total average accuracy of the model
    acc = float(acc)/float(N)
    return model, acc

# DO NOT CHANGE THE CODE AFTER (Assignment) 3.2 Make finetune function 

## 3.2 Train! 

In [61]:
epochs=5
model, acc =finetune(epochs, dataloader, model, loss_fn, optimizer)



  0%|          | 0/217 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,

100%|██████████| 217/217 [00:20<00:00, 10.48it/s]

100%|███

In [62]:
accuracy = acc
print(accuracy)

0.5815092165898618
